In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import requests
import json

# API Keys
from config import gkey
from config import api_key

In [2]:
#Google Places Types: https://developers.google.com/places/web-service/supported_types

#TODO: Can we incorporate this list into the loop so this list can change without changing the code
types = ['bank']#, 'library', 'park', 'liquor_store', 'hospital']

In [3]:
# Read in the Dallas County Appraisal District (DCAD) property values file

file = "Resources/dcad_combined-Copy1.csv"

#create DataFrame
dcad_df = pd.read_csv(file, usecols=['PROPERTY_ZIPCODE', 'TOT_VAL'])
dcad_df.head(5)

,PROPERTY_ZIPCODE,TOT_VAL
0,750513060,103500
1,750513040,145500
2,750502208,168040
3,750502277,200040
4,750617840,151880


In [4]:
#add column for 5 digit zipcode
dcad_df['ZIPCODE'] = dcad_df['PROPERTY_ZIPCODE'].astype(str).str[:5]

In [5]:
#Group Property Values by Zipcode
zip_group = dcad_df.groupby('ZIPCODE')['TOT_VAL']

zip_group = zip_group.mean().to_frame('TOT_VAL')

#Sort the Zipcodes by propert values so we can easily get the top/bottom
zip_group.sort_values(by='TOT_VAL', ascending=False, inplace=True)

zip_group_df = zip_group.reset_index()

In [6]:
#Add columns to DataFrame to store business data
zip_group_df['Lat'] = "" 
zip_group_df["Lng"] = ""
zip_group_df["City"] = ""
zip_group_df["State"] = ""
zip_group_df = zip_group_df.rename(columns={"ZIPCODE": "Zipcode"})
zip_group_df["MeanPropertyValue"] = zip_group_df["TOT_VAL"].map("${:,.0f}".format)

In [7]:
#Preview 5 Zipcodes with highest values
zip_group_df.head(5)

,Zipcode,TOT_VAL,Lat,Lng,City,State,MeanPropertyValue
0,75205,1.606362e+06,,,,,"$1,606,362"
1,75225,1.207824e+06,,,,,"$1,207,824"
2,75201,9.667580e+05,,,,,"$966,758"
3,75209,7.230552e+05,,,,,"$723,055"
4,75230,7.046378e+05,,,,,"$704,638"


In [8]:
#Get 5 Zipcodes with lowest values
zip_group_df.tail(5)

,Zipcode,TOT_VAL,Lat,Lng,City,State,MeanPropertyValue
86,75215,53040.148984,,,,,"$53,040"
87,76065,44860.000000,,,,,"$44,860"
88,75125,42103.223140,,,,,"$42,103"
89,75247,35345.000000,,,,,"$35,345"
90,75210,33010.142772,,,,,"$33,010"


In [9]:
# create a params dict that will be updated with new zipcode each iteration
params = {"key": gkey}

# Loop through the zipcode pd's and run a lat/long search for each
for index, row in zip_group_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    zipcode = row['Zipcode']

    # update address key value to zipcode
    params['address'] = zipcode

    # make request
    zips_lat_lng = requests.get(base_url, params=params)
    
    # convert to json
    zips_lat_lng = zips_lat_lng.json()
    #print(json.dumps(zips_lat_lng, indent=4, sort_keys=True))
    
    #loop through address to find city
    for i in zips_lat_lng['results'][0]['address_components']:
        if i['types'][0] == 'locality':
            zip_group_df.loc[index, "City"] = i['long_name']
    
    #loop though address to find state
    for i in zips_lat_lng['results'][0]['address_components']:
        if i['types'][0] == 'administrative_area_level_1':
            zip_group_df.loc[index, "State"] = i['short_name']
    zip_group_df.loc[index, "Lat"] = zips_lat_lng["results"][0]["geometry"]["location"]["lat"]
    zip_group_df.loc[index, "Lng"] = zips_lat_lng["results"][0]["geometry"]["location"]["lng"]

# Visualize to confirm lat lng appear
zip_group_df

,Zipcode,TOT_VAL,Lat,Lng,City,State,MeanPropertyValue
0,75205,1.606362e+06,32.8326,-96.7976,Dallas,TX,"$1,606,362"
1,75225,1.207824e+06,32.8695,-96.7896,Dallas,TX,"$1,207,824"
2,75201,9.667580e+05,32.7863,-96.7963,Dallas,TX,"$966,758"
3,75209,7.230552e+05,32.8539,-96.819,Dallas,TX,"$723,055"
4,75230,7.046378e+05,32.9005,-96.7869,Dallas,TX,"$704,638"
5,75252,6.376967e+05,33.0009,-96.7976,Dallas,TX,"$637,697"
6,75229,6.056789e+05,32.8958,-96.8726,Dallas,TX,"$605,679"
7,75220,6.025323e+05,32.8622,-96.8726,Dallas,TX,"$602,532"
8,75214,5.574637e+05,32.8243,-96.744,Dallas,TX,"$557,464"
9,75039,4.344581e+05,32.8917,-96.9478,Irving,TX,"$434,458"


In [15]:
# params dictionary to update each iteration
for each_type in types:
    params = {
        #3 mi radius. A Zipcode is not returned in the results, so we cannot 
        #match against our zipcode without doing a reverse lookup for every result
        "radius": 4828,
        "types": each_type,
        "keyword": "bank",
        "key": gkey
    }
    
    #variables for the specific column names for the business type we are searching
    count_column = f"{each_type}_count"
    rating_column = f"{each_type}_rating"
    
    #add columns for each type we looking up
    zip_group_df[count_column] = ""
    zip_group_df[rating_column] = ""


    # Use the lat/lng we recovered to search for businesses
    for index, row in zip_group_df.iterrows():

        rating_sum = 0
        rating_count = 0

        # get lat, lng from df
        lat = row["Lat"]
        lng = row["Lng"]

        # change location each iteration while leaving original params in place
        #params["location"] = f"32.8326,-96.7976" #testing 1 lat, lng
        params["location"] = f"{lat},{lng}"

        #Google Places Search
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        # make request and print url
        search_results = requests.get(base_url, params=params)

        # convert to json
        search_results = search_results.json()
        #print(json.dumps(search_results, indent=4, sort_keys=True))

        results_list = search_results['results']

        #Set the business count
        zip_group_df.loc[index, count_column] = len(results_list)

        #Loop through results list to get rating for each 
        for each_result in results_list:
            #check for KeyError since not all business have a rating
            try:
                #print(f'{each_result["name"]}: {each_result["rating"]}')
                rating = each_result["rating"]
                rating_count += 1
                rating_sum += rating
            except(KeyError):
                next
        #Set Rating to 0 if there are not businesses returned
        try:
            zip_group_df.loc[index, rating_column] = rating_sum / rating_count
        except(ZeroDivisionError):
            zip_group_df.loc[index, rating_column] = 0
            
        detail_df = pd.DataFrame.from_dict(results_list, orient='columns')

In [11]:
zip_group_df.sort_values('TOT_VAL')

,Zipcode,TOT_VAL,Lat,Lng,City,State,MeanPropertyValue,supermarket_count,supermarket_rating,bank_count,bank_rating
90,75210,3.301014e+04,32.7654,-96.7414,Dallas,TX,"$33,010",11,3.85,15,2.95
89,75247,3.534500e+04,32.8167,-96.8834,Dallas,TX,"$35,345",3,3.7,20,3.35455
88,75125,4.210322e+04,32.5204,-96.6424,Ferris,TX,"$42,103",1,4.2,1,3.7
87,76065,4.486000e+04,32.4805,-96.9639,Midlothian,TX,"$44,860",2,4,9,3.675
86,75215,5.304015e+04,32.7557,-96.7655,Dallas,TX,"$53,040",11,3.8,20,3.63571
85,75216,5.710002e+04,32.7031,-96.8029,Dallas,TX,"$57,100",16,3.7125,15,2.91333
84,75172,6.372479e+04,32.6028,-96.6799,Wilmer,TX,"$63,725",1,4.4,2,0
83,75203,7.551809e+04,32.7433,-96.7869,Dallas,TX,"$75,518",17,3.85882,20,3.42941
82,75217,8.055630e+04,32.71,-96.6745,Dallas,TX,"$80,556",15,3.69231,8,3.1
81,75212,8.415152e+04,32.783,-96.8834,Dallas,TX,"$84,152",7,3.97143,20,3.225


In [14]:
detail_df

,geometry,icon,id,name,opening_hours,photos,place_id,plus_code,rating,reference,scope,types,user_ratings_total,vicinity
0,"{'location': {'lat': 32.7919939, 'lng': -96.75...",https://maps.gstatic.com/mapfiles/place_api/ic...,01c6e87ec2e501cf9a92baa785fa6ee022dd67f5,Comerica Bank,{'open_now': False},"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJpbNqGKeYToYR9Ndtsx0ru9c,"{'compound_code': 'Q6RX+Q6 Dallas, Texas, Unit...",3.7,ChIJpbNqGKeYToYR9Ndtsx0ru9c,GOOGLE,"[bank, finance, point_of_interest, establishment]",7.0,"5201 East R L Thornton Freeway, Dallas"
1,"{'location': {'lat': 32.7942926, 'lng': -96.77...",https://maps.gstatic.com/mapfiles/place_api/ic...,26514c2cb79223143fb2948d72a51e59e312a141,Capital One Bank,{'open_now': False},"[{'height': 600, 'html_attributions': ['<a hre...",ChIJAeqZV8-YToYR0RSvdcgUAiM,"{'compound_code': 'Q6VF+P7 Dallas, Texas, Unit...",3.5,ChIJAeqZV8-YToYR0RSvdcgUAiM,GOOGLE,"[bank, atm, finance, point_of_interest, establ...",42.0,"4111 Gaston Avenue, Dallas"
2,"{'location': {'lat': 32.77402159999999, 'lng':...",https://maps.gstatic.com/mapfiles/place_api/ic...,9ca588698ca732251456df5d742a14e6c7cd8ab3,Bank of America Financial Center,{'open_now': False},"[{'height': 1080, 'html_attributions': ['<a hr...",ChIJC7uOSJCYToYRWGB0ixkYswU,"{'compound_code': 'Q6FQ+J4 Dallas, Texas, Unit...",2.6,ChIJC7uOSJCYToYRWGB0ixkYswU,GOOGLE,"[bank, finance, point_of_interest, establishment]",13.0,"3300 Martin Luther King Junior Boulevard, Dallas"
3,"{'location': {'lat': 32.7932226, 'lng': -96.77...",https://maps.gstatic.com/mapfiles/place_api/ic...,57e5dcb086c5c914c6d6ba864379a3398bdce1c1,Bank of America Financial Center,{'open_now': False},"[{'height': 2322, 'html_attributions': ['<a hr...",ChIJveAS5SYV6YARdjgOfTuM4RI,"{'compound_code': 'Q6VC+7M Dallas, Texas, Unit...",2.2,ChIJveAS5SYV6YARdjgOfTuM4RI,GOOGLE,"[bank, finance, point_of_interest, establishment]",34.0,"3921 Gaston Avenue, Dallas"
4,"{'location': {'lat': 32.7703131, 'lng': -96.74...",https://maps.gstatic.com/mapfiles/place_api/ic...,0dcd9b2dff5b72b85acfc3d854b958b63eed6f78,Hibernia Investments,NaN,NaN,ChIJoess-niiToYRSI5shragdnM,"{'compound_code': 'Q7C3+44 Dallas, Texas, Unit...",NaN,ChIJoess-niiToYRSI5shragdnM,GOOGLE,"[bank, atm, finance, point_of_interest, establ...",NaN,Dallas
5,"{'location': {'lat': 32.77022600000001, 'lng':...",https://maps.gstatic.com/mapfiles/place_api/ic...,bb5b410c8e9d117cfe7f405c56cd0df9d6e956f7,Chase Bank,{'open_now': False},"[{'height': 1266, 'html_attributions': ['<a hr...",ChIJC2Rnr42YToYRpsPrtHIABuA,"{'compound_code': 'Q6CJ+3V Dallas, Texas, Unit...",3.7,ChIJC2Rnr42YToYRpsPrtHIABuA,GOOGLE,"[bank, atm, finance, point_of_interest, establ...",18.0,"2833 Martin Luther King Junior Boulevard, Dallas"
6,"{'location': {'lat': 32.748293, 'lng': -96.759...",https://maps.gstatic.com/mapfiles/place_api/ic...,4f255fc4a57ae3bd18c5afabfe166ce026254437,Mjatm Services,NaN,NaN,ChIJ6ynJ1W-YToYRL0JqVMJ47gI,"{'compound_code': 'P6XR+83 Dallas, Texas, Unit...",NaN,ChIJ6ynJ1W-YToYRL0JqVMJ47gI,GOOGLE,"[bank, atm, finance, point_of_interest, establ...",NaN,"4911 South Central Expressway, Dallas"
7,"{'location': {'lat': 32.7786584, 'lng': -96.71...",https://maps.gstatic.com/mapfiles/place_api/ic...,01d73e693bd7d740a56ef85ec3c66f4c7dc46e3b,Carib express envio d dineroe,NaN,NaN,ChIJS6MRfeCjToYRDI-rEYnMnYk,"{'compound_code': 'Q7HP+F2 Dallas, Texas, Unit...",NaN,ChIJS6MRfeCjToYRDI-rEYnMnYk,GOOGLE,"[bank, finance, point_of_interest, establishment]",NaN,Dallas
8,"{'location': {'lat': 32.7934433, 'lng': -96.74...",https://maps.gstatic.com/mapfiles/place_api/ic...,ce44ccf68f511d9b8c114597a6211a08023b5715,"ATM 7ELEVEN, INC.",NaN,NaN,ChIJJ31zTAiiToYRv2yTNmSVLxk,"{'compound_code': 'Q7V3+93 Dallas, Texas, Unit...",NaN,ChIJJ31zTAiiToYRv2yTNmSVLxk,GOOGLE,"[bank, finance, point_of_interest, establishment]",NaN,"5550 East Grand Avenue, Dallas"
9,"{'location': {'lat': 32.7811506, 'lng': -96.77...",https://maps.gstatic.com/mapfiles/place_api/ic...,6f5aca2581c6b5df74